In [36]:
import os
from chainbase_sdk.sql_alpha import ChainbaseSQLAlpha

CHAINBASE_API_KEY=os.environ['CHAINBASE_API_KEY']

In [37]:
client = ChainbaseSQLAlpha(api_key=CHAINBASE_API_KEY)

In [38]:
def withdrawal_initiated(client, chain):
    query = f"""
    SELECT 
        block_number, 
        block_timestamp, 
        transaction_hash, 
        'SentMessage' AS event, 
        signature,
        CAST(decoded AS JSON) -> '$[0]' AS nonce,
        CAST(decoded AS JSON) -> '$[1]' AS sender,
        CAST(decoded AS JSON) -> '$[2]' AS target,
        CAST(decoded AS JSON) -> '$[3]' AS value,
        CAST(decoded AS JSON) -> '$[6]' AS withdrawal_hash
    FROM 
        {chain}.transaction_logs_decoded
    WHERE 
        contract_address = '0x4200000000000000000000000000000000000016'
        and CAST(decoded AS JSON) -> '$[3]' <> 0
    """

    df_l2_withdrawal_initiated = client.query_pandas(query)

    return df_l2_withdrawal_initiated

In [39]:
def withdrawal_proven(client, contract_address):
    query = f"""
    SELECT 
        block_number, 
        block_timestamp, 
        transaction_hash, 
        method_id, 
        'WithdrawalProven' AS event, 
        signature, 
        CAST(decoded AS JSON) -> '$[0]' AS withdrawalHash 
    FROM 
        ethereum.transaction_logs_decoded 
    WHERE 
        contract_address = 'f{contract_address}' 
        AND method_id = '0x67a6208cfcc0801d50f6cbe764733f4fddf66ac0b04442061a8a8c0cb6b63f62' -- WithdrawalProven
    """

    df_l1_proven = client.query_pandas(query)

    return df_l1_proven

In [40]:
def withdrawal_finalized(client, contract_address):
    query = f"""
    SELECT 
        block_number, 
        block_timestamp, 
        transaction_hash, 
        method_id, 
        'WithdrawalFinalized' AS event, 
        signature, 
        CAST(decoded AS JSON) -> '$[0]' AS withdrawalHash 
    FROM 
        ethereum.transaction_logs_decoded 
    WHERE 
        contract_address = 'f{contract_address}' 
        AND method_id = '0xdb5c7652857aa163daadd670e116628fb42e869d8ac4251ef8971d9e5727df1b' -- WithdrawalFinalized
    """

    df_l1_finalized = client.query_pandas(query)

    return df_l1_finalized

In [41]:
def deposit(client, contract_address):
    query = f"""
    SELECT 
        block_number, 
        block_timestamp, 
        transaction_hash, 
        method_id, 
        'TransactionDeposited' AS event, 
        signature, 
        CAST(decoded AS JSON) -> '$[0]' AS sender,
        CAST(decoded AS JSON) -> '$[1]' AS target,
        CONV(SUBSTR(CAST(decoded AS JSON) -> '$[3]', 3, 64), 16, 10) AS msg_value
    FROM 
        ethereum.transaction_logs_decoded 
    WHERE 
        contract_address = '{contract_address}' 
        AND method_id = '0xb3813568d9991fc951961fcb4c784893574240a28925604d09fc577c55bb7c32' -- TransactionDeposited
        AND CONV(SUBSTR(CAST(decoded AS JSON) -> '$[3]', 3, 64), 16, 10) <> 0 -- exclude zero value messages
    """
    df_deposit = client.query_pandas(query)

    return df_deposit

In [42]:
base_withdrawals_started = withdrawal_initiated(client=client, chain='base')

NOT EXISTS:  CHAR
NOT EXISTS:  CHAR
NOT EXISTS:  CHAR
NOT EXISTS:  CHAR
NOT EXISTS:  CHAR


In [43]:
base_withdrawal_proven = withdrawal_proven(client=client, contract_address='0x49048044d57e1c92a77f79988d21fa8faf74e97e')

NOT EXISTS:  CHAR


In [44]:
base_withdrawal_finalized = withdrawal_finalized(client=client, contract_address='0x49048044d57e1c92a77f79988d21fa8faf74e97e')

NOT EXISTS:  CHAR


In [45]:
base_deposits = deposit(client=client, contract_address='0x49048044d57e1c92a77f79988d21fa8faf74e97e')

NOT EXISTS:  CHAR
NOT EXISTS:  CHAR
